In [ ]:
from ultralytics import YOLO
import numpy  as np 
import pandas as pd 
from PIL import Image
import geopandas as gpd
import rasterio 
from rasterio.features import shapes
from shapely.ops import unary_union
from shapely.geometry import box
from scipy.ndimage import label
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import cv2
from shapely.geometry import shape
import os  
import folium

In [ ]:
visualization = {'min': 0, 'max': 0.3}

In [ ]:
block_size = 640
 
def open_snimok(raster_path):
    with rasterio.open(raster_path) as src_image:
        image_data_rgb_canal = src_image.read([1, 2, 3])
        
        b2 = src_image.read(1)
        b3 = src_image.read(2)
        b4 = src_image.read(3)
        b8 = src_image.read(4)
        b11 = src_image.read(5)
        b12 = src_image.read(6)
        bounds = src_image.bounds
        
    image_data_rgb_canal = np.clip(image_data_rgb_canal, visualization['min'], visualization['max'])
    image_data_rgb_canal = (image_data_rgb_canal - visualization['min']) / (visualization['max'] - visualization['min'])

    image_data_rgb_canal = (image_data_rgb_canal * 255).astype(np.uint8)
    rgb_image = np.dstack((image_data_rgb_canal[2], image_data_rgb_canal[1], image_data_rgb_canal[0]))
    ndvi = (b8  - b4 ) / (b8 + b4  )
    ndwi = (b3  - b8 ) / (b3 + b8  )
    turbidity_index = b4 / b8
    
    return  rgb_image,ndvi,ndwi, turbidity_index,bounds


In [ ]:
block_size = 640

def predict_image(rgb_image): 
    final_mask = np.full((rgb_image.shape[0], rgb_image.shape[1]), -1, dtype=np.int32)

    for row in range(0, rgb_image.shape[0], block_size):
        for col in range(0, rgb_image.shape[1], block_size):
            block_image = rgb_image[row:row+block_size, col:col+block_size]
            pil_image = Image.fromarray(block_image)
            
            new_results = model.predict(pil_image, conf=0.2, verbose=False)
            new_result = new_results[0]

            plt.imshow(new_result.plot())
            plt.axis('off')   
            plt.show()
            
            if new_result.masks is not None:
             
                mask_data = new_result.masks.data.cpu().numpy()
                detected_boxes = new_result.boxes.data
                class_labels = detected_boxes[:, -1].int().tolist()

                for mask, class_label in zip(mask_data, class_labels):
                    mask = mask.squeeze()   
                    
                    mask_rows, mask_cols = mask.shape
                    max_row = min(row + mask_rows, rgb_image.shape[0])
                    max_col = min(col + mask_cols, rgb_image.shape[1])
    
                    mask_section = mask[:max_row - row, :max_col - col]
                    final_mask[row:max_row, col:max_col] = np.where(
                        mask_section > 0.5,   
                        class_label,          
                        final_mask[row:max_row, col:max_col]   
                    )

    return final_mask

In [ ]:
def calculate_soil_index(b8, b4):
    soil_index = (b8 - b4) / (b8 + b4)
    soil_index = np.where((b8 + b4) == 0, 0, soil_index)
    return soil_index

In [ ]:
def filter_and_merge_large_objects(binary_image, min_size=1000, dilation_iter=2):
    """
    Оставляет только большие объекты в бинарном изображении и объединяет близко расположенные объекты.

    :param binary_image: Бинарное изображение (numpy array), где объекты имеют значение 1, а фон - 0.
    :param min_size: Минимальный размер объекта в пикселях, который необходимо оставить.
    :param dilation_iter: Количество итераций расширения для объединения близко расположенных объектов.
    
    :return: Маска (numpy array) с большими объединенными объектами.
    """
    # Применяем расширение для объединения близко расположенных объектов
    dilated_image = binary_dilation(binary_image, iterations=dilation_iter)
    
    # Лейблинг (определение связных компонент)
    labeled_array, num_features = label(dilated_image)
    
    # Создаем маску для больших объектов
    large_objects_mask = np.zeros_like(binary_image, dtype=bool)

    for i in range(1, num_features + 1):
        object_slice = labeled_array == i
        if np.sum(object_slice) >= min_size:
            large_objects_mask |= object_slice

    return large_objects_mask

In [ ]:
def count_objects(mask, min_size=2000):
    mask_uint8 =mask.astype(np.uint8) * 255
    
    num_labels, labels_im = cv2.connectedComponents(mask_uint8)
    
    new_mask = np.zeros_like(mask_uint8)
    
    for label in range(1, num_labels): 
        component = (labels_im == label)
        if np.sum(component) >= min_size:
            new_mask[component] = 255
    
    return cv2.connectedComponents(new_mask)[0] - 1, new_mask

def mask_to_polygons(mask, transform, crs):
    geometries = []
    mask_uint8 = (mask > 0).astype(np.uint8)
    
    for geom, value in shapes(mask_uint8, transform=transform):
        if value >= 1:
            poly = shape(geom)
            if poly.geom_type == 'Polygon':
                geometries.append(poly)
            elif poly.geom_type == 'MultiPolygon':
                geometries.extend(poly.geoms)
    
    return gpd.GeoDataFrame(geometry=geometries, crs=crs)

def list_files(directory, extensions=None):

    if extensions is None:
        extensions = ['.tif', '.tiff', '.jpg', '.jpeg', '.png']  
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and os.path.splitext(f)[1].lower() in extensions]

In [ ]:
class DataProcessing:
    def __init__(self, rgb_image, ndvi, ndwi, turbidity, mask):
        self.rgb_image = rgb_image
        self.ndvi = ndvi
        self.ndwi = ndwi
        self.turbidity = turbidity
        self.mask = mask 

    def processing_water(self): 
        only_water = np.zeros_like(self.mask)
        only_water[self.mask == 3] = 3
        turbidity_high = np.copy(only_water)
        only_water[self.ndvi > 0.3] = 0
        ressoil = np.where(only_water, self.turbidity, 0)
        mask_high_soil = ressoil < 1.2
        turbidity_high[mask_high_soil] = 0
        return only_water, turbidity_high

    def processing_gold(self):
        only_gold = np.zeros_like(self.mask)
        only_gold[self.mask == 1] = 1
        only_gold[self.ndwi > 0.3] = 0
        couny_large_gold_objects, large_gold_objects = count_objects(only_gold, min_size=1500)
        return large_gold_objects

In [ ]:
class Violations:
    def __init__(self, rgb_image, ndvi, ndwi, turbidity, mask, gold_mining_mask, boundary_geojson_path, raster_file_path):
        self.rgb_image = rgb_image
        self.ndvi = ndvi
        self.ndwi = ndwi
        self.turbidity = turbidity
        self.mask = mask
        self.gold_mining_mask = gold_mining_mask
        self.boundary_geojson_path = boundary_geojson_path
        self.raster_file_path = raster_file_path
        
    def process_gold_mining(self):
        with rasterio.open(self.raster_file_path) as raster_source:
            raster_bounds = raster_source.bounds
            bounding_box = box(raster_bounds.left, raster_bounds.bottom, raster_bounds.right, raster_bounds.top)
            raster_window = raster_source.window(*bounding_box.bounds)
            raster_band_data = raster_source.read(1, window=raster_window)
            raster_transform = raster_source.window_transform(raster_window)
            raster_crs = raster_source.crs

        self.gold_mining_mask = self.gold_mining_mask.astype(np.uint8)
        boundary_gdf = gpd.read_file(self.boundary_geojson_path).to_crs(raster_crs)
        clipped_boundary_gdf = gpd.clip(boundary_gdf, bounding_box)
        gold_mining_gdf = mask_to_polygons(self.gold_mining_mask, raster_transform, raster_crs)
        # gold_mining_gdf["boundary_violation"] = gold_mining_gdf.intersects(clipped_boundary_gdf.unary_union) & ~gold_mining_gdf.within(clipped_boundary_gdf.unary_union)
        return gold_mining_gdf

    def check_ecological_violations(self, gold_mining_mask, high_turbidity_mask):
        with rasterio.open(self.raster_file_path) as raster_source:
            raster_bounds = raster_source.bounds
            bounding_box = box(raster_bounds.left, raster_bounds.bottom, raster_bounds.right, raster_bounds.top)
            raster_window = raster_source.window(*bounding_box.bounds)
            raster_band_data = raster_source.read(1, window=raster_window)
            raster_transform = raster_source.window_transform(raster_window)
            raster_crs = raster_source.crs

        high_turbidity_gdf = mask_to_polygons(high_turbidity_mask, raster_transform, raster_crs)
        gold_mining_gdf = mask_to_polygons(gold_mining_mask, raster_transform, raster_crs)

        target_crs = 'EPSG:32633'
        high_turbidity_gdf = high_turbidity_gdf.to_crs(target_crs)
        gold_mining_gdf = gold_mining_gdf.to_crs(target_crs)

        # Буферизация и пространственное соединение
        buffer_distance_m = 1000  # Расстояние буфера в метрах
        buffered_gold_mining_gdf = gold_mining_gdf.copy()
        buffered_gold_mining_gdf['geometry'] = buffered_gold_mining_gdf.buffer(buffer_distance_m)
        spatial_joined_gdf = gpd.sjoin(buffered_gold_mining_gdf, high_turbidity_gdf, how='inner', predicate='intersects')
        nearby_gold_mining_gdf = gold_mining_gdf.loc[gold_mining_gdf.index.isin(spatial_joined_gdf.index)]

        # Проверка на пустой GeoDataFrame
        if nearby_gold_mining_gdf.empty:
            print("Нет объектов золотодобычи рядом с зонами высокой мутности.")
            return

        nearby_gold_mining_gdf = nearby_gold_mining_gdf.to_crs(raster_crs)

        print(f'Количество объектов золотодобычи рядом с зонами высокой мутности: {len(nearby_gold_mining_gdf)}')
        return nearby_gold_mining_gdf


In [ ]:
directory_path = r"directory_path"

In [ ]:
model = YOLO(r"best.pt")

In [ ]:
all_gold_border_gdf = gpd.GeoDataFrame()
all_infringing_objects_gdf = gpd.GeoDataFrame()
all_water_gdf = gpd.GeoDataFrame()
all_high_turbidity_water_gdf = gpd.GeoDataFrame()

raster_files = sorted(list_files(directory_path))

for raster_file_path in raster_files:
    # Открытие растра и получение метаданных
    with rasterio.open(raster_file_path) as raster_src:
        raster_bounds = raster_src.bounds
        bounding_box = box(raster_bounds.left, raster_bounds.bottom, raster_bounds.right, raster_bounds.top)
    
        raster_window = raster_src.window(*bounding_box.bounds)
        raster_band_data = raster_src.read(1, window=raster_window)
        raster_transform = raster_src.window_transform(raster_window)
        raster_crs = raster_src.crs

    # Открытие и обработка изображений
    rgb_image, ndvi, ndwi, turbidity, image_bounds = open_snimok(raster_file_path)  

    # Предсказание маски
    prediction_mask = predict_image(rgb_image)  

    # Обработка данных
    data_processor = DataProcessing(rgb_image, ndvi, ndwi, turbidity, prediction_mask)
    gold_mining_mask = data_processor.processing_gold()

    water_mask, high_turbidity_mask = data_processor.processing_water()

    # Обнаружение нарушений
    violations_detector = Violations(
        rgb_image=rgb_image,
        ndvi=ndvi,
        ndwi=ndwi,
        turbidity=turbidity,
        mask=prediction_mask,
        gold_mining_mask=gold_mining_mask,
        boundary_geojson_path="data.geojson",
        raster_file_path=raster_file_path
    )

    gold_border_gdf = violations_detector.process_gold_mining()
    infringing_objects_gdf = violations_detector.check_ecological_violations(gold_mining_mask, high_turbidity_mask)

    # Добавление результатов в общие GeoDataFrame
    all_gold_border_gdf = pd.concat([all_gold_border_gdf, gold_border_gdf], ignore_index=True)
    all_infringing_objects_gdf = pd.concat([all_infringing_objects_gdf, infringing_objects_gdf], ignore_index=True)

    # Преобразование масок в GeoDataFrame
    water_gdf = mask_to_polygons(water_mask, raster_transform, raster_crs)
    high_turbidity_water_gdf = mask_to_polygons(high_turbidity_mask, raster_transform, raster_crs)

    all_water_gdf = pd.concat([all_water_gdf, water_gdf], ignore_index=True)
    all_high_turbidity_water_gdf = pd.concat([all_high_turbidity_water_gdf, high_turbidity_water_gdf], ignore_index=True)

 

In [ ]:
file_path = "data.geojson"
geo_data = gpd.read_file(file_path).to_crs("EPSG:4326")
geo_data_union = geo_data.unary_union

In [ ]:
# Выполнение пространственного соединения между объектами, нарушающими экологию, и границами золотодобычи
joined_violations_gdf = gpd.sjoin(
    all_infringing_objects_gdf,
    all_gold_border_gdf,
    how="left",
    predicate="intersects"
)
# Создание столбца "Environmental Violation", где значение True для строк, пересекающихся с границами
joined_violations_gdf["Environmental Violation"] = joined_violations_gdf.index_right.notnull()

# Выбор только нужных столбцов для окончательного результата
final_violations_gdf = joined_violations_gdf[['Environmental Violation'] + list(all_gold_border_gdf.columns)]

# Установка False для всех строк, где значение "Environmental Violation" является NaN
final_violations_gdf["Environmental Violation"] = final_violations_gdf["Environmental Violation"].fillna(False)

# Приведение столбца "Environmental Violation" к типу bool
final_violations_gdf["Environmental Violation"] = final_violations_gdf["Environmental Violation"].astype(bool)

# Находим объекты, которые есть в all_gold_border_gdf, но отсутствуют в joined_violations_gdf
missing_gold_borders_gdf = all_gold_border_gdf[~all_gold_border_gdf.index.isin(joined_violations_gdf.index_right)]

# Устанавливаем "Environmental Violation" как False для отсутствующих объектов
missing_gold_borders_gdf["Environmental Violation"] = False

# Объединяем основной результат с отсутствующими объектами
final_violations_with_missing_gdf = pd.concat(
    [final_violations_gdf, missing_gold_borders_gdf],
    ignore_index=True
)
# Создание столбца "Border Violation" на основе пересечений с объединёнными геоданными границ
final_violations_with_missing_gdf["Border Violation"] = (
    final_violations_with_missing_gdf.intersects(geo_data_union) & 
    ~final_violations_with_missing_gdf.within(geo_data_union)
)
# Фильтрация только тех записей, где произошло экологическое нарушение
combined_nearby_violations_gdf = final_violations_with_missing_gdf[
    final_violations_with_missing_gdf["Environmental Violation"] == True
]

In [ ]:
center_geometry = final_violations_with_missing_gdf.geometry.unary_union.centroid
map_center_coordinates = [center_geometry.y, center_geometry.x]
dirty_water_gdf = all_high_turbidity_water_gdf

# Функция для добавления объектов на карту
def add_features_to_map(folium_map, geodataframe, color, feature_group=None, add_popup=False):
    """
    Добавляет геометрические объекты из GeoDataFrame на карту Folium.

    :param folium_map: Объект карты Folium.
    :param geodataframe: GeoDataFrame с геометрическими объектами.
    :param color: Цвет заливки объектов.
    :param feature_group: (Опционально) FeatureGroup для добавления объектов.
    :param add_popup: (Опционально) Добавлять ли всплывающее окно с информацией.
    :return: Обновлённый объект карты Folium.
    """
    for _, row in geodataframe.iterrows():
        geometry = row['geometry']
        if geometry is not None:
            geo_json = geometry.__geo_interface__
            fill_color = color

            if add_popup:
                object_name = row.get('license', 'Без названия')  # Получаем имя объекта или используем дефолт
                tooltip = folium.Tooltip(object_name)

                # Создаем всплывающее окно с дополнительной информацией
                additional_info = row.get('Название', 'Нет информации')  # Замените 'Название' на соответствующий столбец
                popup_html = f"""
                <b>Название:</b> {object_name}<br>
                <b>Дополнительная информация:</b> {additional_info}
                """
                popup = folium.Popup(popup_html, max_width=2650)

                feature = folium.GeoJson(
                    geo_json,
                    style_function=lambda x: {
                        'fillColor': fill_color,
                        'color': fill_color,
                        'weight': 1,
                        'fillOpacity': 0.5
                    },
                    tooltip=tooltip,
                    popup=popup
                )
            else:
                feature = folium.GeoJson(
                    geo_json,
                    style_function=lambda x: {
                        'fillColor': fill_color,
                        'color': fill_color,
                        'weight': 1,
                        'fillOpacity': 0.5
                    }
                )
            if feature_group:
                feature.add_to(feature_group)
            else:
                feature.add_to(folium_map)
    return folium_map

# Создание карты Folium
map_object = folium.Map(location=map_center_coordinates, zoom_start=12, tiles=None)

# Создание FeatureGroup для различных слоёв
feature_group_violations = folium.FeatureGroup(name='Нарушения границ участка', show=True)
feature_group_gold_mining = folium.FeatureGroup(name='Объекты золотодобычи', show=True)
feature_group_water = folium.FeatureGroup(name='Водные объекты', show=True)
feature_group_borders = folium.FeatureGroup(name='Границы участков', show=True)
feature_group_pollution_sources = folium.FeatureGroup(name='Источники загрязнения', show=True)
feature_group_dirty_water = folium.FeatureGroup(name='Загрязнённая вода', show=False)
 
# Разделяем GeoDataFrame на объекты с нарушениями и без
violations_present_gdf = final_violations_with_missing_gdf[final_violations_with_missing_gdf['Border Violation'] == True]
violations_absent_gdf = final_violations_with_missing_gdf[final_violations_with_missing_gdf['Border Violation'] == False]

# Добавление слоёв на карту с использованием функции `add_features_to_map`
map_object = add_features_to_map(map_object, all_water_gdf, "blue", feature_group=feature_group_water)
map_object = add_features_to_map(map_object, violations_present_gdf, "red", feature_group=feature_group_violations)
map_object = add_features_to_map(map_object, combined_nearby_violations_gdf, "black", feature_group=feature_group_pollution_sources)
map_object = add_features_to_map(map_object, all_gold_border_gdf, "#ff43a4", feature_group=feature_group_gold_mining)
map_object = add_features_to_map(map_object, geo_data, "#94c3a1", feature_group=feature_group_borders, add_popup=True)
map_object = add_features_to_map(map_object, dirty_water_gdf, "#bb8a5b", feature_group=feature_group_dirty_water)

# Добавляем FeatureGroup на карту
map_object.add_child(feature_group_violations)
map_object.add_child(feature_group_gold_mining)
map_object.add_child(feature_group_water)
map_object.add_child(feature_group_borders)
map_object.add_child(feature_group_pollution_sources)
map_object.add_child(feature_group_dirty_water)

# Создание легенды
legend_html = """
    <div style="
        position: fixed; 
        bottom: 50px; 
        left: 50px; 
        width: 250px; 
        height: auto; 
        background-color: white; 
        border:2px solid grey; 
        z-index:9999; 
        font-size:14px; 
        padding: 10px;">
    <b>Legend:</b><br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: blue;"></span> Чистая вода<br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: #bb8a5b;"></span> Загрязнённая вода<br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: #ff43a4;"></span> Места добычи золота<br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: #94c3a1;"></span> Границы<br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: red;"></span> Нарушения границ<br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: black;"></span> Источники загрязнения<br>

    </div>
"""

map_object.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl().add_to(map_object)

folium.TileLayer(
    tiles='http://{s}.tile.stamen.com/toner/{z}/{x}/{y}.png',
    attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
    name='Gray Background',
    overlay=False,
    control=False
).add_to(map_object)

map_object.save('map.html')
